In [1]:
# Import Dependency 
import pandas as pd
import numpy as np
import psycopg2
import re # Import regex Regular Expressions
import time
from sqlalchemy import create_engine

In [2]:
# Files to laod
accident_data="https://umausdata.s3.amazonaws.com/US_Accidents_Dec21_updated.csv"
accident_df=pd.read_csv(accident_data)

In [3]:
accident_df.head()

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,Between Sawmill Rd/Exit 20 and OH-315/Olentang...,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,2,2016-02-08 05:56:20,2016-02-08 11:56:20,39.865420,-84.062800,39.865010,-84.048730,0.747,At OH-4/OH-235/Exit 41 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-3,2,2016-02-08 06:15:39,2016-02-08 12:15:39,39.102660,-84.524680,39.102090,-84.523960,0.055,At I-71/US-50/Exit 1 - Accident.,...,False,False,False,False,False,False,Night,Night,Night,Day
3,A-4,2,2016-02-08 06:51:45,2016-02-08 12:51:45,41.062130,-81.537840,41.062170,-81.535470,0.123,At Dart Ave/Exit 21 - Accident.,...,False,False,False,False,False,False,Night,Night,Day,Day
4,A-5,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,At Mitchell Ave/Exit 6 - Accident.,...,False,False,False,False,False,False,Day,Day,Day,Day


In [4]:
# drop the Number column
accident_df.drop("Number", axis=1, inplace=True)
accident_df.shape

(2845342, 46)

In [5]:
# Drop Null rows
accident_df=accident_df.dropna()
accident_df.shape

(2207325, 46)

In [6]:
# Change Start Time Object  to Date 
accident_df["Start_Time"] = pd.to_datetime(accident_df['Start_Time'])

# Add Year column
accident_df["Year"]=pd.DatetimeIndex(accident_df["Start_Time"]).year

In [7]:
# Convert ID attribute to Integer
accident_df['ID']=accident_df['ID'].str[2:]
accident_df['ID']=pd.to_numeric(accident_df['ID'])

In [8]:
# Select random 1000 rows for Mock DataSet
mock_accident_df=accident_df.sample(n=50000)

In [9]:
# Create Locate dataframe "ID","Severity","Start_Time","Year","Start_Lat","Start_Lng","Distance(mi)","Street",
# "City","State","Zipcode"

location_df= accident_df[["ID","Severity","Start_Time","Year","Start_Lat","Start_Lng","Distance(mi)","Street", "City","State","Zipcode"]]
location_df.columns = location_df.columns.str.lower()
location_df.rename(columns = {'distance(mi)':'distance'}, inplace = True)
location_df

C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,id,severity,start_time,year,start_lat,start_lng,distance,street,city,state,zipcode
0,1,3,2016-02-08 00:37:08,2016,40.108910,-83.092860,3.230,Outerbelt E,Dublin,OH,43017
4,5,3,2016-02-08 07:53:43,2016,39.172393,-84.492792,0.500,I-75 S,Cincinnati,OH,45217
7,8,2,2016-02-08 11:51:46,2016,41.375310,-81.820170,0.521,I-71 S,Cleveland,OH,44130
9,10,2,2016-02-08 15:16:43,2016,40.109310,-82.968490,0.826,Outerbelt W,Westerville,OH,43081
10,11,2,2016-02-08 15:43:50,2016,39.192880,-84.477230,0.307,I-75 N,Cincinnati,OH,45216
...,...,...,...,...,...,...,...,...,...,...,...
2845337,2845338,2,2019-08-23 18:03:25,2019,34.002480,-117.379360,0.543,Pomona Fwy E,Riverside,CA,92501
2845338,2845339,2,2019-08-23 19:11:30,2019,32.766960,-117.148060,0.338,I-8 W,San Diego,CA,92108
2845339,2845340,2,2019-08-23 19:00:21,2019,33.775450,-117.847790,0.561,Garden Grove Fwy,Orange,CA,92866
2845340,2845341,2,2019-08-23 19:00:21,2019,33.992460,-118.403020,0.772,San Diego Fwy S,Culver City,CA,90230


In [10]:
# Create Weather dataframe with"ID","Temperature(F)","Wind_Chill(F)","Humidity(%)","Pressure(in)","Visibility(mi)","Wind_Direction","Wind_Speed(mph)","Precipitation(in)",
# "Weather_Condition","Sunrise_Sunset"

weather_df=accident_df[["ID","Temperature(F)","Wind_Chill(F)","Humidity(%)","Pressure(in)","Visibility(mi)","Wind_Direction","Wind_Speed(mph)","Precipitation(in)","Weather_Condition","Sunrise_Sunset"]]
weather_df.columns = weather_df.columns.str.lower()
weather_df.rename(columns={'temperature(f)':'temperature',
                            'wind_chill(f)':'wind_chill',
                            'humidity(%)':'humidity',
                            'pressure(in)':'pressure',
                            'visibility(mi)':'visibility',
                            'wind_Speed(mph)':'wind_speed',
                            'precipitation(in)' : 'precipitation'}, inplace = True) 
weather_df
                           

C:\Users\Public\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,id,temperature,wind_chill,humidity,pressure,visibility,wind_direction,wind_speed(mph),precipitation,weather_condition,sunrise_sunset
0,1,42.1,36.1,58.0,29.76,10.0,SW,10.4,0.00,Light Rain,Night
4,5,37.0,29.8,93.0,29.69,10.0,WSW,10.4,0.01,Light Rain,Day
7,8,33.1,30.0,92.0,29.63,0.5,SW,3.5,0.08,Snow,Day
9,10,32.0,28.7,100.0,29.59,0.5,West,3.5,0.05,Snow,Day
10,11,33.8,29.6,100.0,29.66,3.0,NNW,4.6,0.03,Light Snow,Day
...,...,...,...,...,...,...,...,...,...,...,...
2845337,2845338,86.0,86.0,40.0,28.92,10.0,W,13.0,0.00,Fair,Day
2845338,2845339,70.0,70.0,73.0,29.39,10.0,SW,6.0,0.00,Fair,Day
2845339,2845340,73.0,73.0,64.0,29.74,10.0,SSW,10.0,0.00,Partly Cloudy,Day
2845340,2845341,71.0,71.0,81.0,29.62,10.0,SW,8.0,0.00,Fair,Day


In [11]:
# Create Geography Data Frame "ID","Amenity","Bump","Crossing","Give_Way","Junction","No_Exit","Railway","Roundabout","Station","Stop","Traffic_Calming",
# "Traffic_Signal","Turning_Loop"

geography_df=accident_df[["ID","Amenity","Bump","Crossing","Give_Way","Junction","No_Exit","Railway","Roundabout","Station","Stop","Traffic_Calming","Traffic_Signal","Turning_Loop"]]
geography_df.columns = geography_df.columns.str.lower()
geography_df

,id,amenity,bump,crossing,give_way,junction,no_exit,railway,roundabout,station,stop,traffic_calming,traffic_signal,turning_loop
0,1,False,False,False,False,False,False,False,False,False,False,False,False,False
4,5,False,False,False,False,False,False,False,False,False,False,False,False,False
7,8,False,False,False,False,True,False,False,False,False,False,False,False,False
9,10,False,False,False,False,False,False,False,False,False,False,False,False,False
10,11,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845337,2845338,False,False,False,False,False,False,False,False,False,False,False,False,False
2845338,2845339,False,False,False,False,False,False,False,False,False,False,False,False,False
2845339,2845340,False,False,False,False,True,False,False,False,False,False,False,False,False
2845340,2845341,False,False,False,False,False,False,False,False,False,False,False,False,False


In [20]:
# Save the dataframe Location to CSV file
file_dir = '../DataFile/acci_location.csv'
location_df.to_csv(file_dir,index=False)

In [21]:
# Save the dataframe Weather  to CSV file
file_dir = '../DataFile/acci_weather.csv'
weather_df.to_csv(file_dir,index=False)

In [22]:
# Save the dataframe Geography to CSV file
file_dir = '../DataFile/acci_geography.csv'
geography_df.to_csv(file_dir,index=False)

In [23]:
# Save the mock DataFrame  to CSV file
file_dir = '../DataFile/mock_accident_data.csv'
mock_accident_df.to_csv(file_dir,index=False)

In [24]:
# Save the mock DataFrame  to CSV file
file_dir = '../DataFile/accident_data.csv'
accident_df.to_csv(file_dir,index=False)